In [25]:
## imports
import pandas as pd
import numpy as np
from pulp import *
import os

In [26]:
##Load datasets

# Change the working directory to the notebook's directory
notebook_dir = "C:\\Users\\Anu Zan\\Desktop\\GeorgiaTech Spring 2024\\ISYE 6740 CDA\\project"
os.chdir(notebook_dir)

# Now, define the relative path to the dataset
dataset_path = os.path.join("..", "Dataset")

Household_dataset=pd.read_csv(os.path.join(dataset_path, "SampleOptimizationHouseholdData_02142024.csv"))
Intervention_dataset=pd.read_csv(os.path.join(dataset_path,"SampleOptimizationInterventionData_02142024.csv"))

#removed last column(P_NOTX) since it does not appear in the intervention dataset
Household_dataset=Household_dataset.iloc[:, :-1]

In [27]:
## Data cleaning and conversions
Household_dataset['intervention_eligibility_start'] = pd.to_datetime(Household_dataset['intervention_eligibility_start'])
Household_dataset['intervention_eligibility_end'] = pd.to_datetime(Household_dataset['intervention_eligibility_end'])
Intervention_dataset['MoveInDate'] = pd.to_datetime(Intervention_dataset['MoveInDate'])


In [28]:
project_type_mapping = {
    'PH - Rapid Re-Housing': 'RRH',
    'PH - Permanent Supportive Housing': 'PSH',
    'Transitional Housing': 'TSH'  }

In [29]:
intervention_types = list(project_type_mapping.values())
Household_dataset['ID'] = Household_dataset['ID'].astype(str)

In [30]:
# Pre-calculate probabilities for each household and intervention type
probabilities = {}
for _, row in Household_dataset.iterrows():
    household_id = row['ID']
    for description, abbreviation in project_type_mapping.items():
        prob_column = f'P_{abbreviation}'
        if prob_column in Household_dataset.columns:
            probabilities[(household_id, abbreviation)] = row[prob_column]
        else:
            print(f"Warning: Column {prob_column} does not exist in Household_dataset.")


In [31]:
problem = LpProblem("Maximize_exiting_homelessness", LpMaximize)

In [32]:
# Extract household IDs and intervention types
household_ids = Household_dataset['ID'].unique()
intervention_types = list(project_type_mapping.values())


In [33]:
# Decision variables
x = LpVariable.dicts("assignment", (household_ids, intervention_types), cat='Binary')

#here I add a constraint saying that each intervention type have a capacity of 10
intervention_limits = {intervention_type: 10 for intervention_type in intervention_types}


In [34]:

# The objective function
problem += lpSum([x[household_id][intervention_type] * probabilities[(household_id, intervention_type)]
                  for household_id in household_ids for intervention_type in intervention_types])


In [35]:
# Add constraints for intervention limits
for intervention_type in intervention_types:
    problem += lpSum(x[household_id][intervention_type] for household_id in household_ids) <= intervention_limits[intervention_type]


In [36]:
status = problem.solve()

# Check the status of the solution
print(f"Solution Status: {LpStatus[status]}")


Solution Status: Optimal


In [37]:
# Output the assignment decisions (example: print the first 10 decisions)
sample_count = 0
max_samples = 10
for household_id in household_ids:
    for intervention_type in intervention_types:
        if x[household_id][intervention_type].value() == 1:
            print(f"Household {household_id} is assigned to {intervention_type}")
            sample_count += 1
            if sample_count >= max_samples:
                break
    if sample_count >= max_samples:
        break

Household 11550 is assigned to PSH
Household 11491 is assigned to PSH
Household 7104 is assigned to TSH
Household 4815 is assigned to TSH
Household 5433 is assigned to TSH
Household 4342 is assigned to TSH
Household 1170 is assigned to RRH
Household 3842 is assigned to RRH
Household 7275 is assigned to RRH
Household 3436 is assigned to RRH
